In [14]:
# Importation des modules
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re

# # Définir les options d'affichage pour afficher toutes les lignes sans troncature
# pd.set_option("display.max_rows",None)
# pd.set_option("display.max_colwidth",None)

# Modifier les options d'affichage pour limiter les décimales
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# BDD title ratings

In [2]:
title_rate = pd.read_csv("D:/projet2/data/title.ratings.tsv", sep="\t")

In [3]:
# Affichage des informations générales
print(title_rate.info())
title_rate.sample()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448329 entries, 0 to 1448328
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1448329 non-null  object 
 1   averageRating  1448329 non-null  float64
 2   numVotes       1448329 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 33.1+ MB
None


,tconst,averageRating,numVotes
638592,tt1326023,8.3,11


#### Création du dataframe titleRate_clean

In [4]:
# Création du dataframe titleRate_clean
titleRate_clean = pd.DataFrame(title_rate)

### Gestion des NaN

In [5]:
# Transformer les valeurs \N en NaN
titleRate_clean = titleRate_clean.replace("\\N", np.nan)

In [6]:
# Initialisation des comptes de NaN
nan_counts_per_column = {}
total_nan_counts = 0

# Boucle sur chaque colonne et comptage des NaN
for column in titleRate_clean.columns:
    nan_count = titleRate_clean[column].isna().sum()
    nan_counts_per_column[column] = nan_count
    total_nan_counts += nan_count

# Affichage des résultats
print("Nombre de NaN par colonne:")
for column, count in nan_counts_per_column.items():
    print(f"Colonne {column}: {count} NaN")

print("\nNombre total de NaN dans tout le DataFrame:")
print(total_nan_counts)

Nombre de NaN par colonne:
Colonne tconst: 0 NaN
Colonne averageRating: 0 NaN
Colonne numVotes: 0 NaN

Nombre total de NaN dans tout le DataFrame:
0


### Gestions des doublons

In [7]:
# Initialisation d'un dictionnaire pour stocker le nombre de doublons par colonne
duplicate_counts = {}

# Pour chaque colonne du DataFrame
for col in titleRate_clean.columns:
    # Utilisation de duplicated() pour détecter les doublons
    num_duplicates = titleRate_clean[col].duplicated(keep=False).sum() - titleRate_clean[col].duplicated(keep='first').sum()
    duplicate_counts[col] = num_duplicates

# Affichage du nombre de doublons par colonne
for col, count in duplicate_counts.items():
    print(f"Colonne '{col}' a {count} doublons")

# Conversion en DataFrame pour un affichage plus structuré
duplicate_df = pd.DataFrame.from_dict(duplicate_counts, orient='index', columns=['Number of Duplicates'])
print("\nNombre de doublons par colonne :")
print(duplicate_df)

Colonne 'tconst' a 0 doublons
Colonne 'averageRating' a 91 doublons
Colonne 'numVotes' a 9396 doublons

Nombre de doublons par colonne :
               Number of Duplicates
tconst                            0
averageRating                    91
numVotes                       9396


La colonne de "tconst" n'a pas de doublons, donc on peut considérer que le Dataframe n'as pas de doublons.

#### Gestion des Valeurs Manquantes

In [8]:
# On détecte les COLONNES contenant au moins une valeur manquante
print(titleRate_clean.isna().any(axis = 0).sum())
print()
colonne_NA = titleRate_clean.isna().any(axis = 0)
print(colonne_NA)

0

tconst           False
averageRating    False
numVotes         False
dtype: bool


In [9]:
# On détecte les LIGNES contenant au moins une valeur manquante
print("La somme des valeurs manquantes est : " , titleRate_clean.isna().any(axis = 1).sum())
print()
colonne_NA = titleRate_clean.isna().any(axis = 1)

La somme des valeurs manquantes est :  0



#### Modification des nom de colonnes

In [10]:
# Création du dictionnaire associant les anciens noms aux nouveaux noms de colonnes
dictionnaire_colonne_title_rate = {
    "tconst": "ID_title",
    "averageRating": "average_rating",
    "numVotes": "numVotes"
}

# On renomme les variables grâce à la méthode rename
titleRate_clean.rename(dictionnaire_colonne_title_rate, axis=1, inplace=True)
titleRate_clean

,ID_title,average_rating,numVotes
0,tt0000001,5.7,2058
1,tt0000002,5.7,276
2,tt0000003,6.5,2023
3,tt0000004,5.4,179
4,tt0000005,6.2,2789
...,...,...,...
1448324,tt9916730,7.0,12
1448325,tt9916766,7.1,23
1448326,tt9916778,7.2,36
1448327,tt9916840,7.2,10


In [11]:
# Affichages informations générales
print(titleRate_clean.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1448329 entries, 0 to 1448328
Data columns (total 3 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   ID_title        1448329 non-null  object 
 1   average_rating  1448329 non-null  float64
 2   numVotes        1448329 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 33.1+ MB
None


In [19]:
titleRate_clean.describe()

,average_rating,numVotes
count,1448329.00,1448329.00
mean,6.96,1031.89
std,1.38,17699.87
min,1.00,5.00
25%,6.20,11.00
50%,7.20,26.00
75%,7.90,101.00
max,10.00,2906561.00


In [16]:
titleRate_clean.sample()

,ID_title,average_rating,numVotes
1008988,tt27534094,7.50,11


### 1-Sauvegarder en format CSV

In [17]:
# Une sauvegarde du df avec les valeurs manquantes de la colonne profession non supprimées.
titleRate_clean.to_csv('titleRate_clean.csv', index=False)